# Our data is present in form of directories and sub-directories...and this block will extract face from each image and then convert that data into a single numpy array and store in a compressed file.

In [8]:
from os import listdir
from os.path import isdir
import numpy as np;
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import cv2;

detector = MTCNN()

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160, 3)):
     pixels = cv2.imread(filename)
     pixels = cv2.cvtColor(pixels, cv2.COLOR_BGR2RGB)
     pixels = asarray(pixels)
     results = detector.detect_faces(pixels)
     if(len(results) == 0):
        cv2.imshow('img',pixels);
        cv2.waitKey(0);
        return;
     x1, y1, width, height = results[0]['box']
     x1, y1 = abs(x1), abs(y1)
     x2, y2 = x1 + width, y1 + height
     face = pixels[y1:y2, x1:x2]
     face_array = np.resize(face,required_size)
     return face_array
 
    
# load images and extract faces for all images in a directory
def load_faces(directory):
     faces = [];
     for filename in listdir(directory):
         path = directory + filename
         face = extract_face(path)
         faces.append(face)
     return faces
 
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
     X, y = [],[]
     for subdir in listdir(directory):
         path = directory + '/'+ subdir + '/'
         faces = load_faces(path)
         labels = [subdir for _ in range(len(faces))]
         X.extend(faces)
         y.extend(labels)
     return asarray(X), asarray(y)

trainX, trainy = load_dataset('5-celebrity-faces-dataset/train/')
print(trainX.shape, trainy.shape)
testX, testy = load_dataset('5-celebrity-faces-dataset/val/')
savez_compressed('5-celebrity-faces-dataset.npz', trainX, trainy, testX, testy)

here
here
here
here
here
(93, 160, 160, 3) (93,)
here
here
here
here
here


In [9]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras_facenet import FaceNet

    
def get_embedding(model, face_pixels):
 face_pixels = face_pixels.astype('float32')
 mean, std = face_pixels.mean(), face_pixels.std()
 face_pixels = (face_pixels - mean) / std
 face_pixels=face_pixels.reshape(1,160,160,3);
 yhat = model.embeddings(face_pixels)
 return yhat[0]
 

    
data = load('5-celebrity-faces-dataset.npz',allow_pickle=True)
model = FaceNet() 
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
newTrainX = [];
for face_pixels in trainX:
 embedding = get_embedding(model, face_pixels)
 newTrainX.append(embedding)
newTrainX = asarray(newTrainX)

newTestX = [];
for face_pixels in testX:
 embedding = get_embedding(model, face_pixels)
 newTestX.append(embedding)
newTestX = asarray(newTestX)

savez_compressed('5-celebrity-faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
data = load('5-celebrity-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=23.656, test=20.000


In [11]:
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=23.656, test=20.000
